# **Part 2 Model Deployment**

## Deploy FastAPI

We will create a web app for serving our trained model from last exercise and deploy it to Heroku

**Pre-requisites:**
- Create a github account (https://github.com/join)
- Install git (https://git-scm.com/book/en/v2/Getting-Started-Installing-Git)
- Install Docker (https://docs.docker.com/get-docker/)
- Create a Heroku account (https://signup.heroku.com/)

The steps are:
1.   Setup Repository
2.   Build FastApi app
3.   Deploy to Heroku
4.   Push Changes


### 1. Setup Repository

**[1.1]** Go to a folder of your choice on your computer (where you store projects)

**[1.2]** Create a folder called `api` and go inside the created folder

**[1.3]** Copy the `models` folder from Part 1

In [ ]:
# Go to a folder of your choice on your computer (where you store projects)
cd ~/Projects/adv_dsi/adsi_at2

# Create a folder called api and go inside the created folder
mkdir api
cd api

# Copy the trained model from part 1 into models folder
cp -r ../models .

**[1.4]** Initialise the repo

**[1.5]** Login into Github with your account (https://github.com/) and create a public repo with the name `adsi_at2_nn_api`

**[1.6]** In your local repo `adsi_at2_nn_api`, link it with Github (replace the url with your username)

**[1.7]** Add you changes to git staging area and commit them

**[1.8]** Push your master branch to origin

In [ ]:
"""
# Initialise the repo
git init

# Login into Github with your account (https://github.com/) 
# and create a public repo with the name => adsi_at2_nn_api

# Link repo with Github
git remote add origin git@github.com:CazMayhem/adsi_at2_nn_api
    
# Add you changes to git staging area and commit them
git add .
git commit -m "init"

# Push your master branch to origin
git push https://<insert_pat>@github.com/CazMayhem/adsi_at2_nn_api.git --set-upstream origin master
"""

### 2. Build FastApi app

**[2.1]** Create a new git branch called `fastapi`

In [ ]:
# Create a new git branch called fastapi
git checkout -b fastapi

**[2.2]** Create a file called `requirements.txt` with the following content:

`scikit-learn==0.22.1`

`pandas==0.25.3`

`numpy==1.18.1`

`fastapi==0.63.0`

`uvicorn==0.13.3`

`joblib==1.1.0`



**[2.3]** Create a folder called `app`

In [ ]:
# Create 2 folders called app 
mkdir app

**[2.4]** Create a file called `Dockerfile` with the following content:

`FROM tiangolo/uvicorn-gunicorn-fastapi:python3.7`

`COPY requirements.txt .`

`RUN pip3 install -r requirements.txt`

`COPY ./app /app`

`COPY ./models /models`

`CMD ["gunicorn", "-k", "uvicorn.workers.UvicornWorker", "-c", "/gunicorn_conf.py", "main:app"]`

In [ ]:
# Create a file called Dockerfile - copy/paste & save above content
vi Dockerfile

**[2.5]** Create a file called `main.py` in the `app` folder

In [ ]:
# Create a file called main.py in the app folder
vi app/main.py

**[2.6]** Inside the `main.py` file, import FastAPI from fastapi, JSONResponse from starlette.responses, load from joblib and pandas

**[2.7]** Inside the `main.py` file, instantiate a FastAPI() class and save it into a variable called `app`

**[2.8]** Inside the `main.py` file, load your trained model from `models` folder and save it into a variable called `gmm_pipe`

**[2.9]** Inside the `main.py` file, create a function called `read_root()` that will return a dictionary with `Hello` as key and `World` as value. Add a decorator to it in order to add a GET endpoint to `app` on the root

**[2.10]** Inside the `main.py` file, create a function called `healthcheck()` that will return `GMM Clustering is all ready to go!`. Add a decorator to it in order to add a GET endpoint to `app` on `/health` with status code `200`

**[2.11]** Inside the `main.py` file, create a function called `format_features()` with `genre`,	`age`, `income` and `spending` as input parameters that will return a dictionary with the names of the features as keys and the inpot parameters as lists

**[2.12]** Inside the `main.py` file, Define a function called `predict` with the following logics:
- input parameters: `genre`,	`age`, `income` and `spending`
- logics: format the input parameters as dict, convert it to a dataframe and make prediction with `gmm_pipe`
- output: prediction as json

Add a decorator to it in order to add a GET endpoint to `app` on `/mall/customers/segmentation`

In [ ]:
# Inside the main.py file

# import FastAPI from fastapi, JSONResponse from starlette.responses, load from joblib and pandas
from fastapi import FastAPI
from starlette.responses import JSONResponse
from joblib import load
import pandas as pd

# instantiate a FastAPI() class and save it into a variable called app
app = FastAPI()

# load your trained model from models folder and save it into a variable called gmm_pipe
gmm_pipe = load('../models/pytorch_multi_beer_style.pt')

# create a function called read_root() that will return a dictionary with Hello as key and World as value. Add a decorator to it in order to add a GET endpoint to app on the root
@app.get("/")
def read_root():
    return {"Hello": "World"}

# create a function called healthcheck() that will return GMM Clustering is all ready to go!. Add a decorator to it in order to add a GET endpoint to app on /health with status code 200
@app.get('/health', status_code=200)
def healthcheck():
    return 'GMM Clustering is all ready to go!'

# create a function called format_features() with genre, age, income and spending as input parameters that will return a dictionary with the names of the features as keys and the inpot parameters as lists
def format_features(genre: str,	age: int, income: int, spending: int):
    return {
        'Gender': [genre],
        'Age': [age],
        'Annual Income (k$)': [income],
        'Spending Score (1-100)': [spending]
    }

# define a function called predict
@app.get("/mall/customers/segmentation")
def predict(genre: str,	age: int, income: int, spending: int):
    features = format_features(genre,	age, income, spending)
    obs = pd.DataFrame(features)
    pred = gmm_pipe.predict(obs)
    return JSONResponse(pred.tolist())

**[2.13]** Add you changes to git staging area and commit them

In [ ]:
"""
# Add you changes to git staging area and commit them
git add .
git commit -m "init"
"""

**[2.14]** Build the image from the Dockerfile

In [ ]:
# Build the image from the Dockerfile
docker build -t gmm-fastapi:latest .

**[2.15]** Run the built image with port 8080 mapped to 80

In [ ]:
# Run the built image with port 8080 ma
docker run -dit --rm --name adsi_at2_fastapi -p 8080:80 gmm-fastapi:latest

**[2.15]** Open a browser and navigate through:

http://localhost:8080

http://localhost:8080/health

http://localhost:8080/docs

**[2.16]** In your browser, copy-paste http://localhost:8080/mall/customers/segmentation?genre=Female&age=65&income=38&spending=35

**[2.17]** Stop the docker container

In [ ]:
# Stop the docker container
docker stop adsi_at2_fastapi

### 3. Deploy to Heroku

**[3.1]** Login into heroku via command line

**[3.2]** Create a heroku project via command line

**[3.3]** Create a file called `heroku.yml` with the following content:

<img src='https://drive.google.com/uc?id=1KbVLa2IL48F0aysgpsXTNXUXCgPbQ7dC' width="400" height="150">


In [ ]:
# Login into heroku via command line
heroku login

# Create a heroku project via command line
heroku create

# Create a file called heroku.yml
nano heroku.yml

**[3.4]** Add you changes to git staging area and commit them


In [ ]:
# Add you changes to git staging area and commit them
git add .
git commit -m "heroku_nn"

**[3.5]** Set the stack of your app to `container`

In [ ]:
# Push your change to heroku
git push heroku fastapi:master

**[3.7]** On your browser go to the heroku page https://<project_name>.herokuapp.com/docs

like:

https://fast-temple-40998.herokuapp.com/docs

**[3.8]** On your browser go to the heroku page
https://<project_name>.herokuapp.com/mall/customers/segmentation/params?genre=Female&age=65&income=38&spending=35

like:

https://fast-temple-40998.herokuapp.com/mall/customers/segmentation/params?genre=Female&age=65&income=38&spending=35

### 4.   Push changes

**[4.1]** Push your snapshot to Github

**[4.2]** Check out to the master branch

**[4.3]** Pull the latest updates

**[4.4]** Check out to the `fastapi` branch

**[4.5]** Merge the `master` branch and push your changes

**[4.6]** Go to Github and merge the branch after reviewing the code and fixing any conflict

In [ ]:
"""
# Push your snapshot to Github
git push https://<insert_pat>@github.com/CazMayhem/adsi_at2_nn_api.git

# Check out to the master branch
git checkout master

# Pull the latest updates
git pull https://<insert_pat>@github.com/CazMayhem/adsi_at2_nn_api.git

# Merge the branch pytorch_reg
git checkout fastapi

# Merge the master branch and push your changes, 
# any merge issues use: git merge master --allow-unrelated-histories
git merge master 
git push https://<insert_pat>@github.com/CazMayhem/adsi_at2_nn_api.git

"""